In [ ]:
#SETTINGS

#plot-related:
dpi = 360		#plot dpi (size)
rounding = 4	#rounding for floats in plot text

roundingCode = 20	#rounding for the code-generation

#uncomment ONE of the next two lines (only affects plots)
dataMode = 'Volts'	#ADC values converted to a voltage
#dataMode = 'ADC'	#raw ADC values

adcMaxVal = 1023
adcRefVoltage = 5
mechSensorMeasurementAdj = 220/320				#A voltage divider was used to capture the output signal, this means that the output was actually a bit higher than what was captured by the ADC

#fileToOpen = "YAT-Log-20220414-180847.txt"		#Measurement 1
fileToOpen = "YAT-Log-20220414-181008.txt"		#Measurement 2 (best)
fileDelimiter = ";"

cleanThreshold = 600 / mechSensorMeasurementAdj

degree = 2		#degree for regression


In [ ]:
#read file

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

with open(fileToOpen) as file_name:
    array = np.loadtxt(file_name, dtype='str', delimiter=fileDelimiter)

# print(array)


# remove first set of data (time) from the beginning of each line
# this is the first column of the whole csv file
#array = np.delete(array, 0, 1)

# remove text from first row
# this is the first row in the csv file
array = np.delete(array, 0, 0)

array = array.astype(int)
# print(array)



In [ ]:
#adjust data for the mech. sensor
#linear scaling (voltage divider was used for the data capture)

tempArray = array[:, 1] / mechSensorMeasurementAdj

array[:, 1] = tempArray

In [ ]:
# plot with time axis

if (dataMode == 'Volts'):
    array = array * (adcRefVoltage / adcMaxVal)
    array[:, 0] = array[:, 0] / (adcRefVoltage / adcMaxVal)


# Scatter plot
X_plt = array[:, 0]  # time
y_old_plt = array[:, 1]  # old
y_new_plt = array[:, 2]  # new
fig = plt.figure(dpi=dpi)
plt.scatter(X_plt, y_new_plt, color="orange",
            label="Ausgang elektronischer LMM", zorder=3, s=10)
plt.scatter(X_plt, y_old_plt, color="green",
            label="Ausgang mechanischer LMM", zorder=3, s=10)
plt.xlabel("t / ADC Samples")
if (dataMode == 'Volts'):
    plt.ylabel("Ausgangsspannung / V")
else:
    plt.ylabel("ADC-Wert / Bit")
plt.title("Rohe Messdaten")
plt.legend()
plt.grid()
plt.show()


fig.savefig('plots\plot_old_and_new_time.png', dpi=fig.dpi)

In [ ]:
#ab hier für Regression / Korrelation



with open(fileToOpen) as file_name:
    array = np.loadtxt(file_name, dtype='str', delimiter=fileDelimiter)

# print(array)


# remove first set of data (time) from the beginning of each line
# this is the first column of the whole csv file
array = np.delete(array, 0, 1)

# remove text from first row
# this is the first row in the csv file
array = np.delete(array, 0, 0)

array = array.astype(int)

In [ ]:
#adjust data for the mech. sensor
#linear scaling (voltage divider was used for the data capture)

tempArray = array[:, 0] / mechSensorMeasurementAdj

array[:, 0] = tempArray

In [ ]:
#create two arrays, one is only needed for the last step of the regression

arrayADC = array
arrayVolt = array * (adcRefVoltage / adcMaxVal)

if (dataMode == 'Volts'):
    array = arrayVolt
else:
    array = arrayADC

This scatter plot shows the general correlation between the data.
It assumes that the data for the old sensor is in column 0 and the new sensor data in column 1

In [ ]:
# Scatter plot
X_plt = array[:, 1]  # new
y_plt = array[:, 0]  # old

fig = plt.figure(dpi=dpi)
plt.scatter(X_plt, y_plt, zorder=3, s=10)
if (dataMode == 'Volts'):
    plt.ylabel("Ausgang mechanischer LMM / V")
    plt.xlabel("Ausgang elektronischer LMM / V")
else:
    plt.ylabel("Ausgang mechanischer LMM / ADC-Wert")
    plt.xlabel("Ausgang elektronischer LMM / ADC-Wert")
plt.title("Rohe Messdaten")
plt.grid()
plt.show()


fig.savefig('plots\plot_sw_lmm_data_raw.png', dpi=fig.dpi)


In [ ]:
# manually remove data below deleteThreshold_OldData for old sensor readings

deleteThreshold_OldData_Volt = cleanThreshold*adcRefVoltage/adcMaxVal # rows that contain measurements from the old sensor below this value are deleted

deleteThreshold_OldData_ADC = cleanThreshold # rows that contain measurements from the old sensor below this value are deleted

# https://stackoverflow.com/questions/45427833/deleting-row-in-numpy-array-based-on-condition
# array = array[np.where(array[range(array.shape[0]),0] > 400)] # simpler to understand
arrayADC = arrayADC[arrayADC[:, 0] > deleteThreshold_OldData_ADC]  # same effect as line above
arrayVolt = arrayVolt[arrayVolt[:, 0] > deleteThreshold_OldData_Volt] 

if (dataMode == 'Volts'):
    array = arrayVolt
else:
    array = arrayADC

X_plt = array[:, 1]  # new
y_plt = array[:, 0]  # old

In [ ]:
# Scatter plot
fig = plt.figure(dpi=dpi)
plt.scatter(X_plt, y_plt, zorder = 3, s = 10)
if (dataMode == 'Volts'):
    plt.ylabel("Ausgang mechanischer LMM / V")
    plt.xlabel("Ausgang elektronischer LMM / V")
else:
    plt.ylabel("Ausgang mechanischer LMM / ADC-Wert")
    plt.xlabel("Ausgang elektronischer LMM / ADC-Wert")
plt.title("Bereinigte Messdaten")
plt.grid()
plt.show()

fig.savefig('plots\plot_sw_lmm_data_clean.png', dpi=fig.dpi)


In [ ]:
# linear regression

y = array[:, 0].reshape(-1, 1)
X = array[:, 1].reshape(-1, 1)

regr = linear_model.LinearRegression().fit(X, y)
regr.score(X, y)
print("Coefficients: \n", regr.coef_)


y_pred = regr.predict(X)

#print("Coefficients: \n", regr.get_params())
#print(' coef =' + str(regr.coef_)
#    + '\n intercept =' + str(regr.intercept_)
#    +'\n score ='+ str(regr))

#Build a string containing the equation
regEquationComputerReadable = str(round(regr.coef_[0][0], rounding)) + "*x " + ("+ " if regr.intercept_[0] > 0.0 else "") + str(round(regr.intercept_[0], rounding))
regEquationHumanReadable = regEquationComputerReadable

print("regEquationComputerReadable: " + regEquationComputerReadable)
print("regEquationHumanReadable: " + regEquationHumanReadable)

fig = plt.figure(dpi=dpi)
plt.title("Datenkorrelation: Regression 1. Grades")
if (dataMode == 'Volts'):
    plt.ylabel("Ausgang mechanischer LMM / V")
    plt.xlabel("Ausgang elektronischer LMM / V")
else:
    plt.ylabel("Ausgang mechanischer LMM / ADC-Wert")
    plt.xlabel("Ausgang elektronischer LMM / ADC-Wert")
plt.scatter(X, y, label = 'Messdaten', zorder = 3, s = 10)
plt.plot(X, y_pred, color ='maroon', label =  'Regression 1. Grades:\n' + regEquationHumanReadable, zorder = 3)
plt.legend()
plt.grid()
plt.show()

fig.savefig('plots\plot_sw_reg_lmm_data_1_degree.png', dpi=fig.dpi)


In [ ]:
# polynomial regression
# https://towardsdatascience.com/polynomial-regression-with-scikit-learn-what-you-should-know-bed9d3296f2

import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge


polyreg = make_pipeline(PolynomialFeatures(degree, include_bias=False), Ridge())#make_pipeline(PolynomialFeatures(degree), LinearRegression(), Ridge())
polyreg.fit(X, y)

polyreg.predict(X)

#print("Coefficients: \n", polyreg.get_params())
#print(' coef =' + str(polyreg.named_steps['ridge'].coef_)
#    + '\n intercept =' + str(polyreg.named_steps['ridge'].intercept_)
#    +'\n score ='+ str(polyreg))

#dpi = 360

#Build a string containing the equation
regEquationComputerReadable = ""
regEquationHumanReadable = ""

for exponent in range (1,degree+1):
    regEquationComputerReadable += (" + " if (polyreg.named_steps['ridge'].coef_[0][degree-exponent] > 0.0 and degree-exponent != degree-1) else " ") + str(round(polyreg.named_steps['ridge'].coef_[0][degree-exponent], rounding)) + "*x^" + str(degree-exponent+1)
    regEquationHumanReadable += (' + ' if (polyreg.named_steps['ridge'].coef_[0][degree-exponent] > 0.0 and degree-exponent != degree-1) else ' ') + str(round(polyreg.named_steps['ridge'].coef_[0][degree-exponent], rounding)) + '*x$^' + str(degree-exponent+1) + '$'
    
#now add the constant part to the equation
regEquationComputerReadable += ' + ' + str(round(polyreg.named_steps['ridge'].intercept_[0], rounding))
regEquationHumanReadable += ' + ' + str(round(polyreg.named_steps['ridge'].intercept_[0], rounding))

print("regEquationComputerReadable: " + regEquationComputerReadable)
print("regEquationHumanReadable: " + regEquationHumanReadable)

fig = plt.figure(dpi=dpi)
plt.title("Datenkorrelation: Regression " + str(degree) + ". Grades")
if (dataMode == 'Volts'):
    plt.ylabel("Ausgang mechanischer LMM / V")
    plt.xlabel("Ausgang elektronischer LMM / V")
else:
    plt.ylabel("Ausgang mechanischer LMM / ADC-Wert")
    plt.xlabel("Ausgang elektronischer LMM / ADC-Wert")
#plt.text(Any, Any, "test")
plt.scatter(X, y, label = 'Messdaten', zorder = 3, s = 10)
plt.plot(X, polyreg.predict(X), color="maroon", label = 'Regression '+ str(degree) + '. Grades:\n' + regEquationHumanReadable, zorder = 3)
plt.legend()
plt.grid(zorder=-1)
plt.show()

fig.savefig('plots\plot_sw_reg_lmm_data.png', dpi=fig.dpi)



In [ ]:
#calculate coefficients for use in the firmware
#for now, this assumes a regression with degree = 2

#the functional principle is as follows:
#the ADC samples the new, electronic sensor -> ADC-value
#it then applies the coefficients and calculates a desired output -> Volt
#the DAC and OpAmp output this voltage

#this means, that the regression has to be done with two separate units to avoid extra calculations in the microcontroller (ADC value and Volt)

rounding = 5

y = arrayVolt[:, 0].reshape(-1, 1)
X = arrayADC[:, 1].reshape(-1, 1)

polyreg = make_pipeline(PolynomialFeatures(degree, include_bias=False), Ridge())#make_pipeline(PolynomialFeatures(degree), LinearRegression(), Ridge())
polyreg.fit(X, y)

polyreg.predict(X)

#Build a string containing the equation
regEquationComputerReadable = ""
regEquationHumanReadable = ""

for exponent in range (1,degree+1):
    regEquationComputerReadable += (" + " if (polyreg.named_steps['ridge'].coef_[0][degree-exponent] > 0.0 and degree-exponent != degree-1) else " ") + str(round(polyreg.named_steps['ridge'].coef_[0][degree-exponent], rounding)) + "*x^" + str(degree-exponent+1)
    regEquationHumanReadable += (' + ' if (polyreg.named_steps['ridge'].coef_[0][degree-exponent] > 0.0 and degree-exponent != degree-1) else ' ') + str(round(polyreg.named_steps['ridge'].coef_[0][degree-exponent], rounding)) + '*x$^' + str(degree-exponent+1) + '$'
    
#now add the constant part to the equation
regEquationComputerReadable += ' + ' + str(round(polyreg.named_steps['ridge'].intercept_[0], rounding))
regEquationHumanReadable += ' + ' + str(round(polyreg.named_steps['ridge'].intercept_[0], rounding))

print("regEquationComputerReadable: " + regEquationComputerReadable)
print("regEquationHumanReadable: " + regEquationHumanReadable)

fig = plt.figure(dpi=dpi)
plt.title("Datenkorrelation: Regression " + str(degree) + ". Grades")
plt.ylabel("Ausgang mechanischer LMM / V")
plt.xlabel("Ausgang elektronischer LMM / ADC-Wert")
#plt.text(Any, Any, "test")
plt.scatter(X, y, label = 'Messdaten', zorder = 3, s = 10)
plt.plot(X, polyreg.predict(X), color="maroon", label = 'Regression '+ str(degree) + '. Grades:\n' + regEquationHumanReadable, zorder = 3)
plt.legend()
plt.grid(zorder=-1)
plt.show()

fig.savefig('plots\plot_sw_reg_code.png', dpi=fig.dpi)



coefXSquared = polyreg.named_steps['ridge'].coef_[0][1]
coefXOne = polyreg.named_steps['ridge'].coef_[0][0]

coeffOffset = polyreg.named_steps['ridge'].intercept_[0]



#output code that can be copied into the header

print('#define DAC_COEFF_X2 ' + str(round(coefXSquared, roundingCode)))
print('#define DAC_COEFF_X1 ' + str(round(coefXOne, roundingCode)))
print('#define DAC_COEFF_OFFSET ' + str(round(coeffOffset, roundingCode)))

